# 前処理大全 SQL+Python版 (Chapter10)

## はじめに
- データベースはPostgreSQL13です
- 初めに以下のセルを実行してください
- セルに %%sql と記載することでSQLを発行することができます
- jupyterからはdescribeコマンドによるテーブル構造の確認ができないため、テーブル構造を確認する場合はlimitを指定したSELECTなどで代用してください
- 使い慣れたSQLクライアントを使っても問題ありません（接続情報は以下の通り）
  - IPアドレス：Docker Desktopの場合はlocalhost、Docker toolboxの場合は192.168.99.100
  - Port:5432
  - database名: dsdojo_db
  - ユーザ名：padawan
  - パスワード:padawan12345
- 大量出力を行うとJupyterが固まることがあるため、出力件数は制限することを推奨します（設問にも出力件数を記載）
    - 結果確認のために表示させる量を適切にコントロールし、作業を軽快にすすめる技術もデータ加工には求められます
- 大量結果が出力された場合は、ファイルが重くなり以降開けなくなることもあります
    - その場合、作業結果は消えますがファイルをGitHubから取り直してください
    - vimエディタなどで大量出力範囲を削除することもできます
- 名前、住所等はダミーデータであり、実在するものではありません

In [215]:
%load_ext sql
import os

pgconfig = {
    'host': 'db',
    'port': os.environ['PG_PORT'],
    'database': os.environ['PG_DATABASE'],
    'user': os.environ['PG_USER'],
    'password': os.environ['PG_PASSWORD'],
}
dsl = 'postgresql://{user}:{password}@{host}:{port}/{database}'.format(**pgconfig)

# MagicコマンドでSQLを書くための設定
%sql $dsl

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


'Connected: padawan@dsdojo_db'

In [216]:
import pandas as pd
import numpy as np
from scipy.sparse import csc_matrix
import datetime
customer_df = pd.read_csv("./data/customer.csv")
reserve_df = pd.read_csv("./data/reserve.csv")
production_df = pd.read_csv("./data/production.csv")
production_missin_df = pd.read_csv("./data/production_missing_num.csv")
production_missin_category_df = pd.read_csv("./data/production_missing_category.csv")

# 演習問題

## 冒頭

- 今日は日時型の話。

### 010_datetime/001: 日時型、日付型への変換

- 型がいろいろある。TimestampやDateTime。
- UNIXTIMEなどの場合もあり、この場合は日次型に変換する必要がある。
- 日時型以外にも日付型がある。時刻型などもあるが本書では取り扱わない。

<span style="background-color:yellow;">問１：reserveテーブルのreserve_datetimeやcheckin_dateなどを日次型と日付型に変換せよ</span>

#### SQL (Awesome)

- 文字列を入力として、TO_TIMESTAMPおよびTO_DATEを使う
- TO_TIMESTAMPはtimestamptz型になるため、timestampを取得する際はキャストする

In [217]:
%%sql
select
    cast(reserve_datetime as varchar)
    , to_timestamp(
        cast(reserve_datetime as varchar), 'YYYY-MM-DD HH24:MI:SS'
    ) as reserve_datetime_timestamptz
    , cast(
        to_timestamp(
            cast(reserve_datetime as varchar), 'YYYY-MM-DD HH24:MI:SS'
        ) as timestamp
    ) as reserve_datetime_timestamp
    , to_timestamp(
        cast(checkin_date as varchar) || checkin_time, 'YYYY-MM-DDHH24:MI:SS'
    ) as checkin_timestamptz
    , to_date(
        cast(reserve_datetime as varchar), 'YYYY-MM-DD HH24:MI:SS'
    ) as reserve_date
    , to_date(
        cast(checkin_date as varchar), 'YYYY-MM-DD'
    ) as checkin_date
from
    reserve_tb
limit
    10

 * postgresql://padawan:***@db:5432/dsdojo_db
10 rows affected.


reserve_datetime,reserve_datetime_timestamptz,reserve_datetime_timestamp,checkin_timestamptz,reserve_date,checkin_date
2016-03-06 13:09:42,2016-03-06 13:09:42+00:00,2016-03-06 13:09:42,2016-03-26 10:00:00+00:00,2016-03-06,2016-03-26
2016-07-16 23:39:55,2016-07-16 23:39:55+00:00,2016-07-16 23:39:55,2016-07-20 11:30:00+00:00,2016-07-16,2016-07-20
2016-09-24 10:03:17,2016-09-24 10:03:17+00:00,2016-09-24 10:03:17,2016-10-19 09:00:00+00:00,2016-09-24,2016-10-19
2017-03-08 03:20:10,2017-03-08 03:20:10+00:00,2017-03-08 03:20:10,2017-03-29 11:00:00+00:00,2017-03-08,2017-03-29
2017-09-05 19:50:37,2017-09-05 19:50:37+00:00,2017-09-05 19:50:37,2017-09-22 10:30:00+00:00,2017-09-05,2017-09-22
2017-11-27 18:47:05,2017-11-27 18:47:05+00:00,2017-11-27 18:47:05,2017-12-04 12:00:00+00:00,2017-11-27,2017-12-04
2017-12-29 10:38:36,2017-12-29 10:38:36+00:00,2017-12-29 10:38:36,2018-01-25 10:30:00+00:00,2017-12-29,2018-01-25
2018-05-26 08:42:51,2018-05-26 08:42:51+00:00,2018-05-26 08:42:51,2018-06-08 10:00:00+00:00,2018-05-26,2018-06-08
2016-03-05 13:31:06,2016-03-05 13:31:06+00:00,2016-03-05 13:31:06,2016-03-25 09:30:00+00:00,2016-03-05,2016-03-25
2016-06-25 09:12:22,2016-06-25 09:12:22+00:00,2016-06-25 09:12:22,2016-07-14 11:00:00+00:00,2016-06-25,2016-07-14


#### R (Awesome)

- 日時型にPOSIXct型とPOSIXlt型の２種類、日付型はDate型。
- dplyrで処理できるPOSIXct型が推奨。
- いろんな関数で実施しているが、lubridateのparse_date_time2やfast_strptimeが速い様子。
- strptimeはPythonでも見る名前。

#### Python (Awesome)

- datetime64型を利用すれば十分。to_datetime関数で変換できる。
- 日付は、.dt.dateで取得できる。

In [218]:
reserve_df = reserve_df.astype(str)

df = pd.DataFrame([])
df['reserve_datetime'] = pd.to_datetime(reserve_df['reserve_datetime'], format= '%Y-%m-%d %H:%M:%S')
df['reserve_date'] = df['reserve_datetime'].dt.date
df.head(10)

,reserve_datetime,reserve_date
0,2016-03-06 13:09:42,2016-03-06
1,2016-07-16 23:39:55,2016-07-16
2,2016-09-24 10:03:17,2016-09-24
3,2017-03-08 03:20:10,2017-03-08
4,2017-09-05 19:50:37,2017-09-05
5,2017-11-27 18:47:05,2017-11-27
6,2017-12-29 10:38:36,2017-12-29
7,2018-05-26 08:42:51,2018-05-26
8,2016-03-05 13:31:06,2016-03-05
9,2016-06-25 09:12:22,2016-06-25


### 010_datetime/002:年月日などへの変換

- 日時型から要素を取り出す操作。

<span style="background-color:yellow;">問２：reserve_datetimeから年・月・日・時・分・秒を取得せよ。</span>

#### SQL (Awesome)

- date_partで、timestampから要素を取得できる。
- postgresqlでは、yearは'year'と文字列にする必要があった。Redshiftはこのままで良いのかな？
- dowは、日曜スタート。

In [220]:
%%sql
with tmp_log as (
    select
        cast(
            to_timestamp(cast(reserve_datetime as varchar), 'YYYY-MM-DD HH24:MI:SS') as timestamp
        ) as reserve_datetime_timestamp
    from
        reserve_tb
)
select
    date_part('year'  , reserve_datetime_timestamp) as reserve_datetime_year
    , date_part('month' , reserve_datetime_timestamp) as reserve_datetime_month
    , date_part('day'   , reserve_datetime_timestamp) as reserve_datetime_day
    , date_part('dow'   , reserve_datetime_timestamp) as reserve_datetime_dow
    , date_part('hour'  , reserve_datetime_timestamp) as reserve_datetime_hour
    , date_part('minute', reserve_datetime_timestamp) as reserve_datetime_minute
    , date_part('second', reserve_datetime_timestamp) as reserve_datetime_second
    , to_char(reserve_datetime_timestamp, 'YYYY-MM-DD HH24:MI:SS') as reserve_datetime_char
from
    tmp_log
limit
    10

 * postgresql://padawan:***@db:5432/dsdojo_db
10 rows affected.


reserve_datetime_year,reserve_datetime_month,reserve_datetime_day,reserve_datetime_dow,reserve_datetime_hour,reserve_datetime_minute,reserve_datetime_second,reserve_datetime_char
2016.0,3.0,6.0,0.0,13.0,9.0,42.0,2016-03-06 13:09:42
2016.0,7.0,16.0,6.0,23.0,39.0,55.0,2016-07-16 23:39:55
2016.0,9.0,24.0,6.0,10.0,3.0,17.0,2016-09-24 10:03:17
2017.0,3.0,8.0,3.0,3.0,20.0,10.0,2017-03-08 03:20:10
2017.0,9.0,5.0,2.0,19.0,50.0,37.0,2017-09-05 19:50:37
2017.0,11.0,27.0,1.0,18.0,47.0,5.0,2017-11-27 18:47:05
2017.0,12.0,29.0,5.0,10.0,38.0,36.0,2017-12-29 10:38:36
2018.0,5.0,26.0,6.0,8.0,42.0,51.0,2018-05-26 08:42:51
2016.0,3.0,5.0,6.0,13.0,31.0,6.0,2016-03-05 13:31:06
2016.0,6.0,25.0,6.0,9.0,12.0,22.0,2016-06-25 09:12:22


#### R (Awesome)

- POSIXct型から要素を取得するのは関数を使用する必要がある
- POSIXlt型は直接変数を参照して取得できる
- ただし以下は注意が必要そう。
> ただし、fast_strptime関数でPOSIXlt型にしている場合
は、wdayは計算されていないので取得することはできません。

#### Python (Awesome)
- dtオブジェクト経由で直接取得できる。
- Awesomeなこぼれ話
> 日本では2016/08/14と年月日を/を使って区切って表
現しますが、これは世界標準ではありません。そのため、デフォルトの日時文字列フォー
マットでも/は使われていません。これは、データ分析に限らず、文章上でも当てはまる
ルールなので海外とのメールのやりとりでは気を付けましょう。

In [221]:
reserve_df = reserve_df.astype(str)

df = pd.DataFrame([])
df['reserve_datetime'] = pd.to_datetime(reserve_df['reserve_datetime'], format= '%Y-%m-%d %H:%M:%S')
df['reserve_year'] = df['reserve_datetime'].dt.year
df['reserve_month'] = df['reserve_datetime'].dt.month
df['reserve_day'] = df['reserve_datetime'].dt.day
df['reserve_dayofweek'] = df['reserve_datetime'].dt.dayofweek
df['reserve_hour'] = df['reserve_datetime'].dt.hour
df['reserve_minute'] = df['reserve_datetime'].dt.minute
df['reserve_second'] = df['reserve_datetime'].dt.second
df['reserve_char'] = df['reserve_datetime'].dt.strftime('%Y-%m-%d %H:%M:%S')

df.head(10)

,reserve_datetime,reserve_year,reserve_month,reserve_day,reserve_dayofweek,reserve_hour,reserve_minute,reserve_second,reserve_char
0,2016-03-06 13:09:42,2016,3,6,6,13,9,42,2016-03-06 13:09:42
1,2016-07-16 23:39:55,2016,7,16,5,23,39,55,2016-07-16 23:39:55
2,2016-09-24 10:03:17,2016,9,24,5,10,3,17,2016-09-24 10:03:17
3,2017-03-08 03:20:10,2017,3,8,2,3,20,10,2017-03-08 03:20:10
4,2017-09-05 19:50:37,2017,9,5,1,19,50,37,2017-09-05 19:50:37
5,2017-11-27 18:47:05,2017,11,27,0,18,47,5,2017-11-27 18:47:05
6,2017-12-29 10:38:36,2017,12,29,4,10,38,36,2017-12-29 10:38:36
7,2018-05-26 08:42:51,2018,5,26,5,8,42,51,2018-05-26 08:42:51
8,2016-03-05 13:31:06,2016,3,5,5,13,31,6,2016-03-05 13:31:06
9,2016-06-25 09:12:22,2016,6,25,5,9,12,22,2016-06-25 09:12:22


### 010_datetime/003:日時差への変換

- 以下はたしかにそう。
> 日時の差分といっても明確に定義を決めなければ、値の意味が分からなくなってしまい
ます。たとえば、12:45:59と12:46:00の分の差分といっても、秒以下を無視して46－45＝
1分と考えるべきなのか、秒以下を考慮して（60－59）／60 ＝0.016666...分と考えるべきな
のかはケースによって異なります。
>ただし、月と年単位は必ず前者で扱います。なぜなら、月も年も長さが一定ではなく、単位として利用できないからです。

- なのでtimedelta64型などはdayまでの情報しか持たない（monthは日付によってことなり、決められないので）

- なので以下、要注意。
> ただし、年／月は、月／日以下の要素を考慮せずに差分を計算し、時／分／秒は単位に
換算して差分を計算しましょう。

<span style="background-color:yellow;">問３：予約テーブルの予約日時とチェックイン日時の差分を計算せよ</span>

#### SQL (Awesome)

- 前節でやったように要素を取り出して演算してもよいが、DATEDIFFを使用する方がAwesome。
- 切り捨てなので要注意。
> 指定した
単位以下の日時要素は切り捨てて計算します（2015年12月31日と2016年1月1日の年差
分は1年、2016年1月1日と2016 年12 月31日の年差分は0年となります）
- postgresqlの場合、datediffがないので、要素を使うか、unixtimeから計算している。

In [222]:
%%sql
with tmp_log as (
    select
        cast(
            to_timestamp(
                cast(reserve_datetime as varchar), 'YYYY-MM-DD HH24:MI:SS'
            ) as timestamp
        ) as reserve_datetime
        , cast(
            to_timestamp(
                cast(checkin_date as varchar) || checkin_time, 'YYYY-MM-DDHH24:MI:SS'
            ) as timestamp
        ) as checkin_datetime
    from
        reserve_tb
)
select
    reserve_datetime
    , checkin_datetime
    , (checkin_datetime - reserve_datetime) as diff
    , date_part('year'  , checkin_datetime) - date_part('year'  , reserve_datetime) as diff_year
    , date_part('month' , checkin_datetime) - date_part('month' , reserve_datetime) as diff_month
    , ( extract(epoch from checkin_datetime) - extract(epoch from reserve_datetime) )
        / (60*60*24) as diff_day
    , ( extract(epoch from checkin_datetime) - extract(epoch from reserve_datetime) ) 
        / (60*60) as diff_hour
    , ( extract(epoch from checkin_datetime) - extract(epoch from reserve_datetime) ) 
        / (60) as diff_minute
    , ( extract(epoch from checkin_datetime) - extract(epoch from reserve_datetime) ) as diff_second
from
    tmp_log
limit
    20

 * postgresql://padawan:***@db:5432/dsdojo_db
20 rows affected.


reserve_datetime,checkin_datetime,diff,diff_year,diff_month,diff_day,diff_hour,diff_minute,diff_second
2016-03-06 13:09:42,2016-03-26 10:00:00,"19 days, 20:50:18",0.0,0.0,19.8682638888888889,476.8383333333333333,28610.300000000000,1716618.000000
2016-07-16 23:39:55,2016-07-20 11:30:00,"3 days, 11:50:05",0.0,0.0,3.4931134259259259,83.8347222222222222,5030.0833333333333333,301805.000000
2016-09-24 10:03:17,2016-10-19 09:00:00,"24 days, 22:56:43",0.0,1.0,24.9560532407407407,598.9452777777777778,35936.716666666667,2156203.000000
2017-03-08 03:20:10,2017-03-29 11:00:00,"21 days, 7:39:50",0.0,0.0,21.3193287037037037,511.6638888888888889,30699.833333333333,1841990.000000
2017-09-05 19:50:37,2017-09-22 10:30:00,"16 days, 14:39:23",0.0,0.0,16.6106828703703704,398.6563888888888889,23919.383333333333,1435163.000000
2017-11-27 18:47:05,2017-12-04 12:00:00,"6 days, 17:12:55",0.0,1.0,6.7173032407407407,161.2152777777777778,9672.9166666666666667,580375.000000
2017-12-29 10:38:36,2018-01-25 10:30:00,"26 days, 23:51:24",1.0,-11.0,26.9940277777777778,647.8566666666666667,38871.400000000000,2332284.000000
2018-05-26 08:42:51,2018-06-08 10:00:00,"13 days, 1:17:09",0.0,1.0,13.0535763888888889,313.2858333333333333,18797.150000000000,1127829.000000
2016-03-05 13:31:06,2016-03-25 09:30:00,"19 days, 19:58:54",0.0,0.0,19.8325694444444444,475.9816666666666667,28558.900000000000,1713534.000000
2016-06-25 09:12:22,2016-07-14 11:00:00,"19 days, 1:47:38",0.0,1.0,19.0747453703703704,457.7938888888888889,27467.633333333333,1648058.000000


#### R (Awesome)

- POSIXct型の引き算でも可能だが、difftime関数を使用した方がAwesome。
- ただし年月の差分はdifftimeが使えないので要素から計算する。
- 月の差分は、年 x 12を足してあげる必要がある。

#### Python (Awesome)
- 引き算してtimedelta64型になるので、そこから要素を取り出せる。
- R同様、年月の差分を計算する時は、要素から計算しないといけない。
- なんか小数点以下が切り捨てられている気がするんだが、要求されていることと合っていないかも。。。

In [223]:
df = pd.DataFrame([])
df['reserve_datetime'] = pd.to_datetime(reserve_df['reserve_datetime'], format= '%Y-%m-%d %H:%M:%S')
df['checkin_datetime'] = pd.to_datetime(\
    reserve_df['checkin_date'] + reserve_df['checkin_time'], format= '%Y-%m-%d%H:%M:%S'\
)

df['diff_year'] = df['checkin_datetime'].dt.year - df['reserve_datetime'].dt.year

df['diff_month'] = (df['checkin_datetime'].dt.year*12 + df['checkin_datetime'].dt.month) \
    - (df['reserve_datetime'].dt.year*12 + df['reserve_datetime'].dt.month)

df['diff_day'] = (df['checkin_datetime'] - df['reserve_datetime']).astype('timedelta64[D]')
df['diff_hour'] = (df['checkin_datetime'] - df['reserve_datetime']).astype('timedelta64[h]')
df['diff_minute'] = (df['checkin_datetime'] - df['reserve_datetime']).astype('timedelta64[m]')
df['diff_second'] = (df['checkin_datetime'] - df['reserve_datetime']).astype('timedelta64[s]')

df.head(20)

,reserve_datetime,checkin_datetime,diff_year,diff_month,diff_day,diff_hour,diff_minute,diff_second
0,2016-03-06 13:09:42,2016-03-26 10:00:00,0,0,19.0,476.0,28610.0,1716618.0
1,2016-07-16 23:39:55,2016-07-20 11:30:00,0,0,3.0,83.0,5030.0,301805.0
2,2016-09-24 10:03:17,2016-10-19 09:00:00,0,1,24.0,598.0,35936.0,2156203.0
3,2017-03-08 03:20:10,2017-03-29 11:00:00,0,0,21.0,511.0,30699.0,1841990.0
4,2017-09-05 19:50:37,2017-09-22 10:30:00,0,0,16.0,398.0,23919.0,1435163.0
5,2017-11-27 18:47:05,2017-12-04 12:00:00,0,1,6.0,161.0,9672.0,580375.0
6,2017-12-29 10:38:36,2018-01-25 10:30:00,1,1,26.0,647.0,38871.0,2332284.0
7,2018-05-26 08:42:51,2018-06-08 10:00:00,0,1,13.0,313.0,18797.0,1127829.0
8,2016-03-05 13:31:06,2016-03-25 09:30:00,0,0,19.0,475.0,28558.0,1713534.0
9,2016-06-25 09:12:22,2016-07-14 11:00:00,0,1,19.0,457.0,27467.0,1648058.0


- ちょっとアレだが...こういうやり方があった。演算前に切り捨てて良い場合は'D'としてもday単位で計算できそう。
  - [https://stackoverflow.com/questions/62088552/python-pandas-period-date-difference-is-in-monthends-how-to-convert-it-into](https://stackoverflow.com/questions/62088552/python-pandas-period-date-difference-is-in-monthends-how-to-convert-it-into)

In [225]:
df['diff_month'] = (\
    df['checkin_datetime'].dt.to_period('M') \
    - df['reserve_datetime'].dt.to_period('M')\
).apply(lambda x: x.n)
df.head(20)

,reserve_datetime,checkin_datetime,diff_year,diff_month,diff_day,diff_hour,diff_minute,diff_second
0,2016-03-06 13:09:42,2016-03-26 10:00:00,0,0,19.0,476.0,28610.0,1716618.0
1,2016-07-16 23:39:55,2016-07-20 11:30:00,0,0,3.0,83.0,5030.0,301805.0
2,2016-09-24 10:03:17,2016-10-19 09:00:00,0,1,24.0,598.0,35936.0,2156203.0
3,2017-03-08 03:20:10,2017-03-29 11:00:00,0,0,21.0,511.0,30699.0,1841990.0
4,2017-09-05 19:50:37,2017-09-22 10:30:00,0,0,16.0,398.0,23919.0,1435163.0
5,2017-11-27 18:47:05,2017-12-04 12:00:00,0,1,6.0,161.0,9672.0,580375.0
6,2017-12-29 10:38:36,2018-01-25 10:30:00,1,1,26.0,647.0,38871.0,2332284.0
7,2018-05-26 08:42:51,2018-06-08 10:00:00,0,1,13.0,313.0,18797.0,1127829.0
8,2016-03-05 13:31:06,2016-03-25 09:30:00,0,0,19.0,475.0,28558.0,1713534.0
9,2016-06-25 09:12:22,2016-07-14 11:00:00,0,1,19.0,457.0,27467.0,1648058.0


### 010_datetime/004:日時型の増減

- 30日前とかいろいろな増減のこと

<span style="background-color:yellow;">問４：予約レコードについて1日、1時間、1分、1秒を加算せよ</span>

#### SQL (Awesome)
- intervalを使用する。

In [226]:
%%sql
with tmp_log as (
    select
        cast(
            to_timestamp(
                cast(reserve_datetime as varchar), 'YYYY-MM-DD HH24:MI:SS'
            ) as timestamp
        ) as reserve_datetime
        , to_date(
            cast(reserve_datetime as varchar), 'YYYY-MM-DD HH24:MI:SS'
        ) as reserve_date
    from
        reserve_tb
)
select
    reserve_datetime
    , reserve_datetime + interval '1 day' as reserve_datetime_1d
from
    tmp_log
limit
    10

 * postgresql://padawan:***@db:5432/dsdojo_db
10 rows affected.


reserve_datetime,reserve_datetime_1d
2016-03-06 13:09:42,2016-03-07 13:09:42
2016-07-16 23:39:55,2016-07-17 23:39:55
2016-09-24 10:03:17,2016-09-25 10:03:17
2017-03-08 03:20:10,2017-03-09 03:20:10
2017-09-05 19:50:37,2017-09-06 19:50:37
2017-11-27 18:47:05,2017-11-28 18:47:05
2017-12-29 10:38:36,2017-12-30 10:38:36
2018-05-26 08:42:51,2018-05-27 08:42:51
2016-03-05 13:31:06,2016-03-06 13:31:06
2016-06-25 09:12:22,2016-06-26 09:12:22


#### R (Awesome)

- lubridateが提供するdays関数などで加算ができる。

#### Python (Awesome)

- datetime.timedeltaを使って加算できる。

In [227]:
df = pd.DataFrame([])
df['reserve_datetime'] = pd.to_datetime(reserve_df['reserve_datetime'], format='%Y-%m-%d %H:%M:%S')
df['reserve_date'] = df['reserve_datetime'].dt.date
df['reserve_datetime_1d'] = df['reserve_datetime'] + datetime.timedelta(days=1)
df.head(10)

,reserve_datetime,reserve_date,reserve_datetime_1d
0,2016-03-06 13:09:42,2016-03-06,2016-03-07 13:09:42
1,2016-07-16 23:39:55,2016-07-16,2016-07-17 23:39:55
2,2016-09-24 10:03:17,2016-09-24,2016-09-25 10:03:17
3,2017-03-08 03:20:10,2017-03-08,2017-03-09 03:20:10
4,2017-09-05 19:50:37,2017-09-05,2017-09-06 19:50:37
5,2017-11-27 18:47:05,2017-11-27,2017-11-28 18:47:05
6,2017-12-29 10:38:36,2017-12-29,2017-12-30 10:38:36
7,2018-05-26 08:42:51,2018-05-26,2018-05-27 08:42:51
8,2016-03-05 13:31:06,2016-03-05,2016-03-06 13:31:06
9,2016-06-25 09:12:22,2016-06-25,2016-06-26 09:12:22


#### 補足: Pythonのdateutil

- 標準パッケージではないが、dateutilにあるrelativedeltaを使うといろんな単位で増減できる。
- （リスト内包表記が必要だが）
- monthなども加算できるが、扱いに注意は必要で、例えば3か月加算は、末尾は１対１対応にならないので注意が必要。

In [228]:
from dateutil.relativedelta import relativedelta

df['reserve_datetime_30d'] = [ i + relativedelta(months=3) for i in df['reserve_datetime']]
df

,reserve_datetime,reserve_date,reserve_datetime_1d,reserve_datetime_30d
0,2016-03-06 13:09:42,2016-03-06,2016-03-07 13:09:42,2016-06-06 13:09:42
1,2016-07-16 23:39:55,2016-07-16,2016-07-17 23:39:55,2016-10-16 23:39:55
2,2016-09-24 10:03:17,2016-09-24,2016-09-25 10:03:17,2016-12-24 10:03:17
3,2017-03-08 03:20:10,2017-03-08,2017-03-09 03:20:10,2017-06-08 03:20:10
4,2017-09-05 19:50:37,2017-09-05,2017-09-06 19:50:37,2017-12-05 19:50:37
...,...,...,...,...
4025,2017-06-27 23:00:02,2017-06-27,2017-06-28 23:00:02,2017-09-27 23:00:02
4026,2017-09-29 05:24:57,2017-09-29,2017-09-30 05:24:57,2017-12-29 05:24:57
4027,2018-03-14 05:01:45,2018-03-14,2018-03-15 05:01:45,2018-06-14 05:01:45
4028,2016-04-16 15:20:17,2016-04-16,2016-04-17 15:20:17,2016-07-16 15:20:17


- 以下は同じ日に割り当てられる

In [230]:
print( datetime.datetime.strptime("2022-01-30 10:00:00", "%Y-%m-%d %H:%M:%S") \
      + relativedelta(months=3) )
print( datetime.datetime.strptime("2022-01-31 10:00:00", "%Y-%m-%d %H:%M:%S") \
      + relativedelta(months=3) )

2022-04-30 10:00:00
2022-04-30 10:00:00


### 010_datetime/005:季節への変換

- アイスクリームを例に、季節に着目するのが良いのか十分に検討が必要であることを述べている。

<span style="background-color:yellow;">問５：予約レコードについて、春夏秋冬の列を追加せよ</span>

#### SQL (Awesome)

- CASE文を使います。

In [231]:
%%sql
with tmp_log as (
    select
        reserve_datetime
        , date_part(
            'month',
            cast(
                to_timestamp(
                    cast(reserve_datetime as varchar), 'YYYY-MM-DD HH24:MI:SS'
                ) as timestamp
            )
        ) as reserve_month
    from
        reserve_tb
)
select
    reserve_datetime
    , case
        when 3<= reserve_month and reserve_month <= 5 THEN 'spring'
        when 6<= reserve_month and reserve_month <= 8 THEN 'summer'
        when 10<= reserve_month and reserve_month <= 11 THEN 'autumn'
        else 'winter'
    end as reserve_seazon
from
    tmp_log
limit
    10

 * postgresql://padawan:***@db:5432/dsdojo_db
10 rows affected.


reserve_datetime,reserve_seazon
2016-03-06 13:09:42,spring
2016-07-16 23:39:55,summer
2016-09-24 10:03:17,winter
2017-03-08 03:20:10,spring
2017-09-05 19:50:37,winter
2017-11-27 18:47:05,autumn
2017-12-29 10:38:36,winter
2018-05-26 08:42:51,spring
2016-03-05 13:31:06,spring
2016-06-25 09:12:22,summer


#### R (Awesome)
- mutate関数でやるみたい。

#### Python (Awesome)
- 変換関数を準備してapplyで変換させる。

In [232]:
df = pd.DataFrame([])
df['reserve_datetime'] = pd.to_datetime(reserve_df['reserve_datetime'], format='%Y-%m-%d %H:%M:%S')

def to_season(month_num: int):
    season = 'winter'
    if 3 <= month_num <= 5:
        season = 'spring'
    elif 6 <= month_num <= 8:
        season = 'summer'
    elif 9 <= month_num <= 11:
        season = 'autumn'
    return season

df['reserve_season'] = pd.Categorical(df['reserve_datetime'].dt.month.apply(to_season))
df.head(10)

,reserve_datetime,reserve_season
0,2016-03-06 13:09:42,spring
1,2016-07-16 23:39:55,summer
2,2016-09-24 10:03:17,autumn
3,2017-03-08 03:20:10,spring
4,2017-09-05 19:50:37,autumn
5,2017-11-27 18:47:05,autumn
6,2017-12-29 10:38:36,winter
7,2018-05-26 08:42:51,spring
8,2016-03-05 13:31:06,spring
9,2016-06-25 09:12:22,summer


### 010_datetime/006:時間帯への変換

- 季節と同様、どのような時間帯にわけるのか注意深く検討が必要
- 基本的には005と同様なコードなので省略

### 010_datetime/007:平日・休日への変換

- 休日や休前日で傾向が変わったりするので重要

<span style="background-color:yellow;">問７：休日フラグを予約レコードについて付与せよ</span>

#### SQL/R/Python (Awesome)

- 休日マスタと結合するのがよい。祝日は種類が少ないし変動が大きい。

### 補足: Pythonの日付型indexによる集計

- indexは嫌われているが、日付単位での集計の際は思い出してほしい。
- [https://note.nkmk.me/python-pandas-time-series-resample-asfreq/](https://note.nkmk.me/python-pandas-time-series-resample-asfreq/)

In [233]:
reserve_df = pd.read_csv("./data/reserve.csv")
reserve_df['reserve_datetime'] \
    = pd.to_datetime(reserve_df['reserve_datetime'], format='%Y-%m-%d %H:%M:%S')
# reserve_df.set_index('reserve_datetime', drop=True)[['total_price']].resample('M').sum()

# と思ったけど、on使えるのでやっぱindexなくてもいい…？
reserve_df.resample('M', on='reserve_datetime')[['total_price']].sum()

,total_price
reserve_datetime,
2016-01-31,16475900
2016-02-29,15527200
2016-03-31,16582200
2016-04-30,22388400
2016-05-31,30254100
2016-06-30,34536200
2016-07-31,18040000
2016-08-31,14913900
2016-09-30,20860000
